<a href="https://colab.research.google.com/github/vaishtml/02_assignment/blob/main/assignment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Question 1 LLM APIs**

In [ ]:
!pip install huggingface_hub
#Inference Client

# Import required modules
from huggingface_hub import InferenceClient
from getpass import getpass
import textwrap

# Define primary and fallback models
PRIMARY_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"
FALLBACK_MODEL = "HuggingFaceH4/zephyr-7b-beta"

# Function to send prompt and get response from the model
def ask_question(model_name, token, prompt):
    client = InferenceClient(model=model_name, token=token)
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    return client.chat_completion(messages=messages, max_tokens=150)

try:
    # Get Hugging Face token securely
    HF_TOKEN = getpass("Enter your Hugging Face token: ")

     # Take user's question
    user_prompt = input("Ask your question: ")

    try:
        # Try using the primary model
        print(f"\nUsing primary model: {PRIMARY_MODEL}")
        response = ask_question(PRIMARY_MODEL, HF_TOKEN, user_prompt)

    except Exception as primary_error:
        # If primary fails, try fallback model
        print(f"Primary model failed: {primary_error}")
        print(f"\nTrying fallback model: {FALLBACK_MODEL}")

        try:
            response = ask_question(FALLBACK_MODEL, HF_TOKEN, user_prompt)
        except Exception as fallback_error:
            # If fallback also fails, show final error
            raise RuntimeError(f"Both primary and fallback models failed:\n{fallback_error}")

    # Print the model's response
    print("\nAssistant Response:\n")
    print(textwrap.fill(response.choices[0].message.content.strip(), width=80))

# General error handling
except Exception as final_error:
    print(f"\nFatal error: {final_error}")

Enter your Hugging Face token: ··········
Ask your question: hello

Using primary model: mistralai/Mistral-7B-Instruct-v0.3

Assistant Response:

Hello! How can I assist you today? If you have any questions or need help with
something, feel free to ask. I'm here to help!


In [14]:
import requests
from getpass import getpass
import json
import textwrap

# Model name
MODEL_NAME = "deepset/roberta-base-squad2"

# Function to call the API
def ask_question_with_context(token, context, question):
    API_URL = f"https://api-inference.huggingface.co/models/{MODEL_NAME}"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    payload = {
        "inputs": {
            "question": question,
            "context": context
        }
    }

    response = requests.post(API_URL, headers=headers, data=json.dumps(payload))

    if response.status_code != 200:
        raise RuntimeError(f"Request failed with status code {response.status_code}: {response.text}")

    return response.json()

# Start of the main flow
try:
    HF_TOKEN = getpass("Enter your Hugging Face token: ")

    context_text = input("\nPaste your context:\n")

    ask_now = input("\nWould you like to ask a question based on this context? (yes/no): ").strip().lower()

    if ask_now == "yes":
        question_text = input("\nType your question:\n")

        if question_text.strip().lower() == "exit":
            print("\nYou chose to exit. Goodbye!")
        else:
            try:
                response = ask_question_with_context(HF_TOKEN, context_text, question_text)
                answer = response.get("answer", "No answer found.")

                print("\nAnswer:\n")
                print(textwrap.fill(answer.strip(), width=80))

            except Exception as model_error:
                print(f"\nModel request failed: {model_error}")

    else:
        print("\n Goodbye!")

except Exception as final_error:
    print(f"\nFatal error: {final_error}")

Enter your Hugging Face token: ··········

Paste your context:
abcdefghijklmnopqrstuvwxyz

Would you like to ask a question based on this context? (yes/no): yes

Type your question:
what comes after a?

Answer:

abcdefghijklmnopqrstuvwxyz


**Question 2**

In [13]:
# Step 1: Install required libraries
!pip install transformers

# Step 2: Import necessary libraries
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Step 3: Load GPT-2 model and tokenizer
try:
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2LMHeadModel.from_pretrained("gpt2")

    # Set pad token to avoid warning
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

    # Use GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

except Exception as e:
    print("Error loading GPT-2 model or tokenizer:", e)

# Step 4: Define the generate_story function
def generate_story(prompt, temperature=0.7):
    """
    Generate a story using GPT-2 based on a given prompt and temperature value.

    Parameters:
        prompt (str): The initial text to start the story.
        temperature (float): Controls randomness (0.3 = deterministic, 1.3 = very creative)

    Returns:
        str: Generated story text
    """
    try:
        # Encode the prompt
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

        # Generate continuation
        output = model.generate(
            input_ids,
            max_length=200,
            do_sample=True,           # Enables sampling
            temperature=temperature,  # Controls creativity
            top_k=50,                 # Limits to top-k probable tokens
            top_p=0.95,               # Limits to top-p cumulative probability
            repetition_penalty=1.2,   # Penalizes repetition
            pad_token_id=tokenizer.eos_token_id  # Avoid warning
        )

        # Decode and return output
        story = tokenizer.decode(output[0], skip_special_tokens=True)
        print(f"\n--- Temperature: {temperature} ---")
        print(story)

    except Exception as e:
        print("Error during story generation:", e)

# Step 5: Test cases with different temperatures
test_prompt = "Once upon a time in a distant galaxy, a young explorer"

# Run the function with various temperature values
for temp in [0.3, 0.7, 1.0, 1.3]:
    generate_story(test_prompt, temperature=temp)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



--- Temperature: 0.3 ---
Once upon a time in a distant galaxy, a young explorer named K'Kai was captured by the evil Emperor of Mankind. He had been sent to Earth for his mission and he soon became one with humanity's most powerful warriors - but not before being forced into battle against an ancient alien race known as "The Shadow."
A new generation has emerged within this world that is now called The Empire...and it will take them all on their own journey through space!

--- Temperature: 0.7 ---
Once upon a time in a distant galaxy, a young explorer called Karrn decided to explore the mysterious moon Serenity. He was given access to an ancient chamber and he spent some of his childhood trying to figure out what happened there when it came down into space!


He went on searching for more secrets but found nothing... until one night while exploring deep within another star system where they met up with two other explorers who had come from different worlds which allowed them both new 